Chapter 2.b.iv.  Instantiation
=======

**Instantiation** is a powerful derivation rule that can transform known truths with great flexibility by implicitly invoking *Lambda application* (beta reduction).  It is based upon the fact that variables can represent anything and, therefore, we may replace each free variable in an existing known truth with any expression, as long as we do so consistently.  That is, we may replace any free variable on either side of the turnstile of a known truth with any expression, as long as we replace all occurrences of the free variable across the known truth in a consistent manner.  Implicitly, if a known truth is valid, any such transformation must be allowed.

To be more precise about the machinery, an ad-hoc Lambda expression is generated for each component of the known truth (each assumption as well as the right side) with the respective component becoming the body of the corresponding Lambda expression.  The parameters of each of these ad-hoc lambda expressions will be the same and are dictacted by which variables are to be replaced.  Then beta reduction is applied using operands dictated by the desired replacements, the same operands for each ad-hoc lambda expression, with all of the reduction rules discussed in previous chapters.  The reduced expressions form a new known truth.  For example, some $\{A(x)\} \vdash P(x)$ where $x$ is a variable can be transformed to some $\{A(t)\} \vdash P(t)$ where $t$ is an arbitrary expression via applying the ad-hoc lambda expressions, $x \mapsto A(x)$ and $x \mapsto P(x)$, to the operand $t$.

*Assumptions* may be provided when performing an instantiation.  These may be needed to satisfy *requirements* needed to validate any reductions when performing the lambda applications.  Such requirements then become requirements of the derivation step, creating additional dependencies (beyond that of the original known truth) in the proof graph.  Assumptions that were needed to fulfill these requirements will be added as assumptions in the new known truth.

Finally, and importantly, instantiation may be used to eliminate any number of outer universal quantifiers on the right side of the turnstile provided that the conditions of the quantifiers are satisfied under these replacements.  These also become *requirements* of the derivation step to be included in the proof graph and may need *assumptions* to be proven.  For example, we can instantiate $A \vdash \forall_{x~|~Q_1(x), Q_2(x)} P(x)$ to get $A \cup B \vdash P(t)$ provided $A \cup B \vdash Q_1(t)$ and $A \cup B \vdash Q_2(t)$ are provable.

In [1]:
import proveit
%begin instantiation

## Instantiation as lambda applications (beta reductions)

First, let's prove a "known truth" that we can work with for demonstrating instantiation.

In [2]:
from proveit._common_ import A, P, Q, R
from proveit.logic import Implies
P_impl_R = Implies(P, R).prove([Implies(P, Q), Implies(Q, R)])

P_impl_R: {P => Q, Q => R} |- P => R

The following is just a simple relabeling via `instantiate`, changing 
the label $P$ to $A$ on both sides of the known truth via supplying `{P:A}` as the `repl_map` dictionary in the call to `instantiate`.

In [3]:
A_impl_R = P_impl_R.instantiate({P:A})

A_impl_R: {A => Q, Q => R} |- A => R

Internally, this is done by effectively generating the lambda expressions $P \mapsto (P \Rightarrow Q)$, $P \mapsto (Q \Rightarrow R)$, and $P \mapsto (P \Rightarrow R)$, applying each of these functions to the single operand $A$, and then replacing those respective beta reductions to the respective components of the original known truth.  The proof reveals the **instantiation** step which is followed by the replacement mapping, $P:A$, and requires the original known truth.

In [4]:
A_impl_R.proof()

step type	requirements	statement
0	instantiation	1	{A => Q, Q => R} |- A => R
	P : A
1	hypothetical reasoning	2	{P => Q, Q => R} |- P => R
2	modus ponens	3, 4	{P => Q, Q => R, P} |- R
3	assumption		{Q => R} |- Q => R
4	modus ponens	5, 6	{P => Q, P} |- Q
5	assumption		{P => Q} |- P => Q
6	assumption		{P} |- P

The following example is done by effectively generating the lambda expressions $(P, R) \mapsto (P \Rightarrow Q)$, $(P, R) \mapsto (Q \Rightarrow R)$, and $(P, R) \mapsto (P \Rightarrow R)$ and applying each of these functions to the operands $(A, A)$.

In [5]:
A_impl_R = P_impl_R.instantiate({P:A, R:A})

A_impl_R: {A => Q, Q => A} |- A => A

We can `instantiate` to any expressions, not just variables.  For example

In [6]:
from proveit import Lambda
from proveit._common_ import f, x, y, fx, fy
from proveit.number import subtract
fx_impl_R_weird = P_impl_R.instantiate({P:fx, Q:Lambda(y, subtract(y, x))})

fx_impl_R_weird: {f(x) => [y -> (y - x)], [y -> (y - x)] => R} |- f(x) => R

This may seem like a silly statement, and it is.  The assumptions are absurd with Lambda expressions for an implication consequent and an implication antecedant that should certainly not evaluate to true or false.  **Prove-It** does not care, however.  It is making no presumption that the implication operands are Boolean values.  It is merely a statement that if you were to assume these absurd things as being true, you can logically derive that $f(x)$ implies $R$.  In Prove-It's universe, this is a fact under these assumptions.  To prove consistency, we need not worry about whether we can derive bogus facts from bogus assumptions (or bogus implication antecedents), we only need to ensure that we cannot derive bogus facts from valid assumptions.

## Instantiating ranges of free variables

In addition to instantiating any free variables of a given known truth, we can also instantiate a range of its free variables.  To demonstrate, let us first create a known truth with a range of free variables.

In [7]:
from proveit.core_expr_types._common_ import x_1_to_n, f__x_1_to_n
f__x_1_to_n__impl_R = P_impl_R.instantiate({P:f__x_1_to_n})

f__x_1_to_n__impl_R: {f(x_{1}, ..x_{_a}.., x_{n}) => Q, Q => R} |- f(x_{1}, ..x_{_a}.., x_{n}) => R

Now let's prepare a replacement for this range of variables.

In [8]:
from proveit import ExprRange, ExprTuple
from proveit.logic import InSet
from proveit._common_ import a, n
from proveit.number import Naturals, Add, one
x_repl_1 = ExprRange(a, Add(a, a), one, n)

x_repl_1: (1 + 1), ..(a + a).., (n + n)

And now we may call `instantiate` to make this replacement.

In [9]:
f__1p1_to_npn__impl_R = f__x_1_to_n__impl_R.instantiate(
    {ExprTuple(x_1_to_n):ExprTuple(x_repl_1)}, assumptions=[InSet(n, Naturals)])

f__1p1_to_npn__impl_R: {f(1 + 1, ..a + a.., n + n) => Q, Q => R, n in Naturals} |- f(1 + 1, ..a + a.., n + n) => R

This was done internally by effectively generating the lambda expressions $(x_1, ..., x_n) \mapsto (f(x_1, .., x_n) \Rightarrow Q)$, $(x_1, ..., x_n) \mapsto (Q \Rightarrow R)$, and $(x_1, ..., x_n) \mapsto (f(x_1, .., x_n) \Rightarrow R)$, applying each of these functions to the operands $(1+1), ..., (n+n)$ under the assumption that $n \in \mathbb{N}$.  $n \in \mathbb{N}$ was added to the assumptions list and the proof reveals the extra requirement necessary to make sure that the number of parameters and operands do indeed match (step 2 of the proof).

In [10]:
f__1p1_to_npn__impl_R.proof()

step type	requirements	statement
0	instantiation	1, 2	{f(1 + 1, ..a + a.., n + n) => Q, Q => R, n in Naturals} |- f(1 + 1, ..a + a.., n + n) => R
	(x_{1}, ..x_{_a}.., x_{n}) : (1 + 1, ..a + a.., n + n)
1	instantiation	3	{f(x_{1}, ..x_{_a}.., x_{n}) => Q, Q => R} |- f(x_{1}, ..x_{_a}.., x_{n}) => R
	P : f(x_{1}, ..x_{_a}.., x_{n})
2	instantiation	4, 5	{n in Naturals} |- |(1 + 1, ..a + a.., n + n)| = |(1, ..k.., n)|
	i : n, f(a) : a + a
3	hypothetical reasoning	6	{P => Q, Q => R} |- P => R
4	theorem		|- forall_{i in Naturals} [forall_{f} (|(f(1), ..f(a).., f(i))| = |(1, ..k.., i)|)]
	proveit.core_expr_types.tuples.range_from1_len_typical_equiv
5	assumption		{n in Naturals} |- n in Naturals
6	modus ponens	7, 8	{P => Q, Q => R, P} |- R
7	assumption		{Q => R} |- Q => R
8	modus ponens	9, 10	{P => Q, P} |- Q
9	assumption		{P => Q} |- P => Q
10	assumption		{P} |- P

As we saw in a previous chapter with respect to lambda applications, we are allowed to split apart this range, supplying multiple operand entries for a single parameter entry as long as their lengths match.

In [11]:
from proveit._common_ import a, k, m, x
from proveit.number import zero, Mult
x_repl_2 = ExprRange(a, Mult(a, a), zero, k), x, ExprRange(a, a, one, m)

x_repl_2: (0 * 0, ..a * a.., k * k, x, 1, ..a.., m)

In [12]:
from proveit.core_expr_types import Len
from proveit.logic import Equals
assumption = Equals(Len(x_repl_2), Len(ExprRange(a, a, one, n)))

assumption: |(0 * 0, ..a * a.., k * k, x, 1, ..a.., m)| = |(1, ..a.., n)|

In [13]:
arbitrary_fn__impl__R = f__x_1_to_n__impl_R.instantiate({ExprTuple(x_1_to_n):x_repl_2}, 
                                                        assumptions=[assumption])

arbitrary_fn__impl__R: {f(0 * 0, ..a * a.., k * k, x, 1, ..a.., m) => Q, Q => R, |(0 * 0, ..a * a.., k * k, x, 1, ..a.., m)| = |(1, ..a.., n)|} |- f(0 * 0, ..a * a.., k * k, x, 1, ..a.., m) => R

### Instantiating with alternative expansions

We also saw in a previous chapter that we may optionally provide an `equiv_alt_expansions` dictionary when calling `Lambda.apply` to provide versatility when a variable occurs in various forms with respect to ranging over its indices.  To access this feature in an **instantiation**, simply provide the alternative expansions in the `repl_map`.  For example, let's convert the following expression (with partial masking of some of the $A$ variable indices) to a known truth by simply assuming it to be true.

In [14]:
from proveit import IndexedVar, varRange
from proveit._common_ import A, B, C, D, i, j, k, m
from proveit.core_expr_types._common_ import A_i_to_j
from proveit.logic import Not, And, Or, Forall
from proveit.number import one, Neg, subtract, NaturalsPos
A_1_to_j, A_m = varRange(A, one, j), IndexedVar(A, m)
B_i_to_j = varRange(B, i, j)
partially_masked_expr = And(A_1_to_j, Forall(A_i_to_j, Or(A_i_to_j)))

partially_masked_expr: A_{1} and ..A_{_a}.. and A_{j} and [forall_{A_{i}, ..A_{_a}.., A_{j}} (A_{i} or ..A_{_a}.. or A_{j})]

In [15]:
partially_masked_known_truth = partially_masked_expr.prove([partially_masked_expr])

partially_masked_known_truth: {A_{1} and ..A_{_a}.. and A_{j} and [forall_{A_{i}, ..A_{_a}.., A_{j}} (A_{i} or ..A_{_a}.. or A_{j})]} |- A_{1} and ..A_{_a}.. and A_{j} and [forall_{A_{i}, ..A_{_a}.., A_{j}} (A_{i} or ..A_{_a}.. or A_{j})]

And now we will prepare `operands` and an alternate form for $A_1, \ldots, A_k$, making some assumptions and proving some equivalences as needed.

In [16]:
operands = ExprTuple(ExprRange(k, Not(IndexedVar(B, k)), one, subtract(i, one)), B_i_to_j)

operands: ([not](B_{1}), ..[not](B_{k}).., [not](B_{i - 1}), B_{i}, ..B_{_a}.., B_{j})

In [17]:
assumptions = [InSet(i, NaturalsPos), InSet(j, NaturalsPos), InSet(Add(j, Neg(i), one), Naturals)]
#InSet(subtract(j, subtract(i, one)), Naturals)]

In [18]:
alt_A_form = ExprTuple(varRange(A, one, subtract(i, one)), A_i_to_j)

alt_A_form: (A_{1}, ..A_{_a}.., A_{i - 1}, A_{i}, ..A_{_a}.., A_{j})

In [19]:
alt_A_form.merger(assumptions)

{i in NaturalsPos, (j - i + 1) in Naturals, j in NaturalsPos} |- (A_{1}, ..A_{_a}.., A_{i - 1}, A_{i}, ..A_{_a}.., A_{j}) = (A_{1}, ..A_{a}.., A_{j})

In [20]:
from proveit import extract_var_tuple_indices
extract_var_tuple_indices(alt_A_form).merger(assumptions)

{i in NaturalsPos, (j - i + 1) in Naturals, j in NaturalsPos} |- (1, .._a.., i - 1, i, .._a.., j) = (1, ..a.., j)

Finally, we will call the `instantiate` method, providing an expansion for $(A_1, \ldots, A_j)$ as well as $(A_1, \ldots A_{i-1}, A_i, \ldots, A_j)$ (we simply use `operands` for both).

In [21]:
partially_masked_known_truth.instantiate({ExprTuple(A_1_to_j):operands, alt_A_form:operands},
                                        assumptions=assumptions)

{[not](B_{1}) and ..[not](B_{_a}).. and [not](B_{i - 1}) and B_{i} and ..B_{_a}.. and B_{j} and [forall_{A_{i}, ..A_{_a}.., A_{j}} (A_{i} or ..A_{_a}.. or A_{j})], i in NaturalsPos, (j - i + 1) in Naturals, j in NaturalsPos} |- [not](B_{1}) and ..[not](B_{_a}).. and [not](B_{i - 1}) and B_{i} and ..B_{_a}.. and B_{j} and [forall_{B_{i}, ..B_{_a}.., B_{j}} (B_{i} or ..B_{_a}.. or B_{j})]

Notice that we relabeled the inner masked portion, replacing the $\forall_{A_i, \ldots, A_j}$ with $\forall_{B_i, \ldots, B_j}$.  Instantiation always does this inner relabeling when it is valid to do so (it doesn't have the `allow_relabeling` flag that the `Lambda.apply` method has, but it isn't really necessary as we will see).  If the replacement for a masked portion is not a proper relabeling, it will simply skip it.

In [22]:
operands = ExprTuple(ExprRange(k, Not(IndexedVar(B, k)), one, subtract(i, one)), 
                     ExprRange(k, Not(IndexedVar(B, k)), i, j))

operands: ([not](B_{1}), ..[not](B_{k}).., [not](B_{i - 1}), [not](B_{i}), ..[not](B_{k}).., [not](B_{j}))

In [23]:
partially_masked_known_truth.instantiate({ExprTuple(A_1_to_j):operands, alt_A_form:operands},
                                        assumptions=assumptions)

{[not](B_{1}) and ..[not](B_{_a}).. and [not](B_{i - 1}) and [not](B_{i}) and ..[not](B_{_a}).. and [not](B_{j}) and [forall_{A_{i}, ..A_{_a}.., A_{j}} (A_{i} or ..A_{_a}.. or A_{j})], i in NaturalsPos, (j - i + 1) in Naturals, j in NaturalsPos} |- [not](B_{1}) and ..[not](B_{_a}).. and [not](B_{i - 1}) and [not](B_{i}) and ..[not](B_{_a}).. and [not](B_{j}) and [forall_{A_{i}, ..A_{_a}.., A_{j}} (A_{i} or ..A_{_a}.. or A_{j})]

This kind of masking should generally be avoided in any case, but if there is no other way around the need to relabel an inner expression, the inner expression can always be substituted with a form that is proven to be equal to it but with different labels.  More typically, we can perform a simple variable relabeling of an inner expression to be discussed in more detail in Chapter 2.a.vi.

In [24]:
partially_masked_known_truth.innerExpr().operands[1].relabel({A:C})

A_{1} and ..A_{_a}.. and A_{j} and [(C_{i}, ..C_{_a}.., C_{j}) -> (C_{i} or ..C_{_a}.. or C_{j})]

### Instantiating operators with lambda expressions

The other reduction rules discussed in previous chapters are also applicable during **instantiation** of course.  For example, if we can now instantiate $f$ with a Lambda expression to invoke a reduction of the $f$ operations via an extra internal beta reduction.

In [25]:
from proveit import Lambda
f_repl = Lambda(x_1_to_n, Add(x_1_to_n))

f_repl: (x_{1}, ..x_{_a}.., x_{n}) -> (x_{1} + ..x_{_a}.. + x_{n})

In [26]:
arbitrary_fn__impl__R.instantiate({f:f_repl}, assumptions=[assumption])

{((0 * 0) + ..(a * a).. + (k * k) + x + 1 + ..a.. + m) => Q, Q => R, |(0 * 0, ..a * a.., k * k, x, 1, ..a.., m)| = |(1, ..a.., n)|} |- ((0 * 0) + ..(a * a).. + (k * k) + x + 1 + ..a.. + m) => R

Once again, we have a silly statement, but it is no less truthful as far as **Prove-It** is concerned.  The right side of the known truth does derive from its silly assumptions.

Rather than using a Lambda expression for the replacement, we may also use the implicit form $f(x_1, .., x_n) : x_1 + ... + x_n$.  This form is preferable, in fact, though internally it is transformed to $f : (x_1, .., x_n) \mapsto x_1 + ... + x_n$ as we had before.

In [27]:
wacky = arbitrary_fn__impl__R.instantiate({f__x_1_to_n:Add(x_1_to_n)}, assumptions=[assumption])

wacky: {((0 * 0) + ..(a * a).. + (k * k) + x + 1 + ..a.. + m) => Q, Q => R, |(0 * 0, ..a * a.., k * k, x, 1, ..a.., m)| = |(1, ..a.., n)|} |- ((0 * 0) + ..(a * a).. + (k * k) + x + 1 + ..a.. + m) => R

Note, however, that the $f(x_1, .., x_n) : x_1 + ... + x_n$ form is used when the proof graph is displayed (see the line below step 0):

In [28]:
wacky.proof()

step type	requirements	statement
0	instantiation	1, 3	{((0 * 0) + ..(a * a).. + (k * k) + x + 1 + ..a.. + m) => Q, Q => R, |(0 * 0, ..a * a.., k * k, x, 1, ..a.., m)| = |(1, ..a.., n)|} |- ((0 * 0) + ..(a * a).. + (k * k) + x + 1 + ..a.. + m) => R
	f(x_{1}, ..x_{_a}.., x_{n}) : x_{1} + ..x_{_a}.. + x_{n}
1	instantiation	2, 3	{f(0 * 0, ..a * a.., k * k, x, 1, ..a.., m) => Q, Q => R, |(0 * 0, ..a * a.., k * k, x, 1, ..a.., m)| = |(1, ..a.., n)|} |- f(0 * 0, ..a * a.., k * k, x, 1, ..a.., m) => R
	(x_{1}, ..x_{_a}.., x_{n}) : (0 * 0, ..a * a.., k * k, x, 1, ..a.., m)
2	instantiation	4	{f(x_{1}, ..x_{_a}.., x_{n}) => Q, Q => R} |- f(x_{1}, ..x_{_a}.., x_{n}) => R
	P : f(x_{1}, ..x_{_a}.., x_{n})
3	assumption		{|(0 * 0, ..a * a.., k * k, x, 1, ..a.., m)| = |(1, ..a.., n)|} |- |(0 * 0, ..a * a.., k * k, x, 1, ..a.., m)| = |(1, ..a.., n)|
4	hypothetical reasoning	5	{P => Q, Q => R} |- P => R
5	modus ponens	6, 7	{P => Q, Q => R, P} |- R
6	assumption		{Q => R} |- Q => R
7	modus ponens	8, 9	{P => Q, P} |- Q
8	assumption		{P => Q} |- P => Q
9	assumption		{P} |- P

### Eliminating universal quantifiers while instantiating

The above examples illustrate what can be done by instantiating free variables on either side of an existing known truth.  A more common and useful application of instantiation, however, is to eliminate universal quantifiers on the right side of an existing known truth.  For example,

In [29]:
from proveit import Function
from proveit._common_ import x, Px, Qx
from proveit.logic import Forall
simple_univ_quant = Forall(x, Px, condition=Qx)

simple_univ_quant: forall_{x | Q(x)} P(x)

To make a convenient example, we will just prove this by assumption.

In [30]:
simple_univ_quant_assumed = simple_univ_quant.prove(assumptions=[simple_univ_quant])

simple_univ_quant_assumed: {forall_{x | Q(x)} P(x)} |- forall_{x | Q(x)} P(x)

In [31]:
simple_univ_quant_inst = simple_univ_quant_assumed.instantiate(
    {x:Add(a, x)}, assumptions=[Function(Q, Add(a, x))])

simple_univ_quant_inst: {forall_{x | Q(x)} P(x), Q(a + x)} |- P(a + x)

Since $x$ was in our `repl_map` and it is universally quantified directly on the right side of the turnstile in `simple_univ_quant_assumed`, it is the indication to the `instantiate` method that we wish to eliminate that universal quantifier.  In the process, we apply or replacement map on the $Q(x)$ condition to obtain $Q(a+x)$ which is a requirement for this derivation step.  In this example, we satisify this requirement by simply assuming it.  In the proof graph below, this requirement is shown explicitly as step 2.

In [32]:
simple_univ_quant_inst.proof()

step type	requirements	statement
0	instantiation	1, 2	{forall_{x | Q(x)} P(x), Q(a + x)} |- P(a + x)
	x : a + x
1	assumption		{forall_{x | Q(x)} P(x)} |- forall_{x | Q(x)} P(x)
2	assumption		{Q(a + x)} |- Q(a + x)

We can eliminate multiple universal quantifiers simultaneously if the are nested directly on the right side of the quantifier.  It will eliminate the nested quantifiers as deep as the deepest of the variables in the `repl_map`.  For example, consider

In [33]:
from proveit._common_ import a, b, c, d
nested_univ_quants = Forall(a, Forall(b, Forall(c, Forall(d, Function(P, (a, b, c, d))))))

nested_univ_quants: forall_{a} [forall_{b} [forall_{c} [forall_{d} P(a, b, c, d)]]]

In [34]:
nested_univ_quants.instantiate({a:x, c:x}, assumptions=[nested_univ_quants])

{forall_{a} [forall_{b} [forall_{c} [forall_{d} P(a, b, c, d)]]]} |- forall_{d} P(x, b, x, d)

We eliminated down to the quantifier for $c$ which was the deepest of the variable in the `repl_map`.

### Eliminating universal quantifiers with ranges of parameters and conditions

When instantiating ranges of parameters in univeral quantifiers being eliminated, the relevant range of indices  is indicated by the quantifier and is therefore not needed explicitly in the `repl_map`.  For example, instead of $(x_1, ..., x_n) : ((1+1), ..., (n+n))$, you can use $x : ((1+1), ..., (n+n))$ when $\forall_{..., x_1, ..., x_n, ...}$ is being eliminated.  This is called an implicit instantiation.  In this next example, let us invoke a theorem that is actually useful rather than something silly.

In [35]:
from proveit.core_expr_types.tuples._theorems_ import general_len

In [36]:
general_len

|- forall_{n in NaturalsPos} [forall_{f_{1}, ..f_{_a}.., f_{n}, i_{1}, ..i_{_a}.., i_{n}, j_{1}, ..j_{_a}.., j_{n} | (j_{1} - i_{1} + 1) in Naturals, ..(j_{k} - i_{k} + 1) in Naturals.., (j_{n} - i_{n} + 1) in Naturals} (|(f_{1}(i_{1}), ..f_{1}(x).., f_{1}(j_{1}), ....f_{k}(i_{k}), ..f_{k}(x).., f_{k}(j_{k})...., f_{n}(i_{n}), ..f_{n}(x).., f_{n}(j_{n}))| \\  = ((j_{1} - i_{1} + 1) + ..(j_{k} - i_{k} + 1).. + (j_{n} - i_{n} + 1)))]

Let us instantiate this for one of our previous example tuples.

In [37]:
x_repl_2

(0 * 0, ..a * a.., k * k, x, 1, ..a.., m)

In [38]:
general_len

|- forall_{n in NaturalsPos} [forall_{f_{1}, ..f_{_a}.., f_{n}, i_{1}, ..i_{_a}.., i_{n}, j_{1}, ..j_{_a}.., j_{n} | (j_{1} - i_{1} + 1) in Naturals, ..(j_{k} - i_{k} + 1) in Naturals.., (j_{n} - i_{n} + 1) in Naturals} (|(f_{1}(i_{1}), ..f_{1}(x).., f_{1}(j_{1}), ....f_{k}(i_{k}), ..f_{k}(x).., f_{k}(j_{k})...., f_{n}(i_{n}), ..f_{n}(x).., f_{n}(j_{n}))| \\  = ((j_{1} - i_{1} + 1) + ..(j_{k} - i_{k} + 1).. + (j_{n} - i_{n} + 1)))]

In [39]:
from proveit.number import num, Neg
from proveit._common_ import i, j
general_len_inst = \
    general_len.instantiate(
    {n:num(3), f:[Lambda(k, Mult(k, k)), Lambda(k, k), Lambda(k, k)],
     i:[zero, one, one], j:[k, one, m]},
    assumptions=[InSet(k, Naturals), InSet(m, Naturals)])

general_len_inst: {k in Naturals, m in Naturals} |- |(0 * 0, ..x * x.., k * k, 1, 1, ..x.., m)| = ((k - 0 + 1) + (1 - 1 + 1) + (m - 1 + 1))

Looking at the proof for this (below), there are a number of requirements that needed to be satisfied for the instantiation.  In addition tot the `general_len_val` theorem itself, we needed to prove length matches for the $f_1, ..., f_3$, $i_1, ..., i_3$, and $j_1, ..., j_3$ replacements, and we needed to prove the $(j_1 - i_1 +1) \in \mathbb{N}, ..., (j_n - i_n +1) \in \mathbb{N}$ conditions.  Finally, because $f_2(i_1), ..., f_2(j_2)$ became $1, ..., 1$ which reduced to simply $1$, we require the fact that $(1, ..., 1) = (1)$ (step 8).

In [40]:
general_len_inst.proof()

step type	requirements	statement
0	instantiation	1, 2, 3, 4, 5, 6, 7, 8, 9	{k in Naturals, m in Naturals} |- |(0 * 0, ..x * x.., k * k, 1, 1, ..x.., m)| = ((k - 0 + 1) + (1 - 1 + 1) + (m - 1 + 1))
	n : 3, f : (k -> (k * k), k -> k, k -> k), i : (0, 1, 1), j : (k, 1, m)
1	theorem		|- forall_{n in NaturalsPos} [forall_{f_{1}, ..f_{_a}.., f_{n}, i_{1}, ..i_{_a}.., i_{n}, j_{1}, ..j_{_a}.., j_{n} | (j_{1} - i_{1} + 1) in Naturals, ..(j_{k} - i_{k} + 1) in Naturals.., (j_{n} - i_{n} + 1) in Naturals} (|(f_{1}(i_{1}), ..f_{1}(x).., f_{1}(j_{1}), ....f_{k}(i_{k}), ..f_{k}(x).., f_{k}(j_{k})...., f_{n}(i_{n}), ..f_{n}(x).., f_{n}(j_{n}))| \\  = ((j_{1} - i_{1} + 1) + ..(j_{k} - i_{k} + 1).. + (j_{n} - i_{n} + 1)))]
	proveit.core_expr_types.tuples.general_len
2	theorem		|- 3 in NaturalsPos
	proveit.number.numeral.deci.posnat3
3	instantiation	10	|- |(k -> (k * k), k -> k, k -> k)| = |(1, ..k.., 3)|
	a(k) : k * k, b(k) : k, c(k) : k
4	instantiation	10	|- |(0, 1, 1)| = |(1, ..k.., 3)|
	a : 0, b : 1, c : 1
5	instantiation	10	|- |(k, 1, m)| = |(1, .._a.., 3)|
	a : k, b : 1, c : m
6	instantiation	14, 11, 12	{k in Naturals} |- (k - 0 + 1) in Naturals
	P(_a) : _a in Naturals, x : k - 0 + 1, y : k + 1
7	instantiation	14, 36, 13	|- (1 - 1 + 1) in Naturals
	P(_a) : _a in Naturals, x : 1 - 1 + 1, y : 1
8	instantiation	14, 31, 15	{m in Naturals} |- (m - 1 + 1) in Naturals
	P(_a) : _a in Naturals, x : m - 1 + 1, y : m
9	instantiation	16	|- (1, ..a.., 1) = (1)
	f(x) : x, i : 1
10	theorem		|- forall_{a, b, c} (|(a, b, c)| = |(1, ..k.., 3)|)
	proveit.number.numeral.deci.tuple_len_3_typical_equiv
11	instantiation	17, 33, 36	{k in Naturals} |- (k + 1) in Naturals
	a : k, b : 1
12	instantiation	18, 19, 20	{k in Naturals} |- (k - 0 + 1) = (k + 1)
	x : k - 0 + 1, y : k + 0 + 1, z : k + 1
13	instantiation	21, 30	|- (1 - 1 + 1) = 1
	a : 1, b : 1
14	theorem		|- forall_{P, x, y | P(y), x = y} P(x)
	proveit.logic.equality.subLeftSideInto
15	instantiation	22, 30, 23	{m in Naturals} |- (m - 1 + 1) = m
	a : 1, b : m
16	theorem		|- forall_{f, i} ((f(i), ..f(a).., f(i)) = (f(i)))
	proveit.core_expr_types.tuples.singular_range_reduction
17	theorem		|- forall_{a in Naturals, b in Naturals} ((a + b) in Naturals)
	proveit.number.addition.addNatClosureBin
18	axiom		|- forall_{x, y, z | x = y, y = z} (x = z)
	proveit.logic.equality.equalsTransitivity
19	instantiation	24, 25	|- (k - 0 + 1) = (k + 0 + 1)
	f(_a) : k + _a + 1, x : -0, y : 0
20	instantiation	26, 36, 27, 28, 29, 30	{k in Naturals} |- (k + 0 + 1) = (k + 1)
	i : 1, j : 1, a : (k), b : (1)
21	theorem		|- forall_{a in Complexes, b in Complexes} ((a - a + b) = b)
	proveit.number.addition.subtraction.addCancelTriple_12
22	theorem		|- forall_{a in Complexes, b in Complexes} ((b - a + a) = b)
	proveit.number.addition.subtraction.addCancelTriple_32
23	instantiation	34, 35, 31	{m in Naturals} |- m in Complexes
	A : Naturals, B : Complexes, x : m
24	axiom		|- forall_{f, x, y | x = y} (f(x) = f(y))
	proveit.logic.equality.substitution
25	theorem		|- (-0) = 0
	proveit.number.negation.negatedZero
26	theorem		|- forall_{i in Naturals, j in Naturals} [forall_{a_{1} in Complexes, ..a_{_a} in Complexes.., a_{i} in Complexes, b_{1} in Complexes, ..b_{_a} in Complexes.., b_{j} in Complexes} ((a_{1} + ..a_{_a}.. + a_{i} + 0 + b_{1} + ..b_{_a}.. + b_{j}) = (a_{1} + ..a_{_a}.. + a_{i} + b_{1} + ..b_{_a}.. + b_{j}))]
	proveit.number.addition.elimZeroAny
27	instantiation	32	|- |(k)| = |(1, .._a.., 1)|
	a : k
28	instantiation	32	|- |(1)| = |(1, ..k.., 1)|
	a : 1
29	instantiation	34, 35, 33	{k in Naturals} |- k in Complexes
	A : Naturals, B : Complexes, x : k
30	instantiation	34, 35, 36	|- 1 in Complexes
	A : Naturals, B : Complexes, x : 1
31	assumption		{m in Naturals} |- m in Naturals
32	theorem		|- forall_{a} (|(a)| = |(1, ..k.., 1)|)
	proveit.number.numeral.deci.tuple_len_1_typical_equiv
33	assumption		{k in Naturals} |- k in Naturals
34	theorem		|- forall_{A, B | A subseteq B} [forall_{x in A} (x in B)]
	proveit.logic.set_theory.cont

Finally, let us check the treatment when some of the instantiated conditions of an eliminated universal quantifiction are contained within an `ExprRange`.  Consider this direct instantiation (eliminating universal quantifiers with no non-trivial replacements) of the `general_len` theorem.

In [41]:
general_len_simple_inst = \
    general_len.instantiate({j:j}, assumptions=general_len.allConditions())

general_len_simple_inst: {n in NaturalsPos, (j_{1} - i_{1} + 1) in Naturals, ..(j_{k} - i_{k} + 1) in Naturals.., (j_{n} - i_{n} + 1) in Naturals} |- |(f_{1}(i_{1}), ..f_{1}(x).., f_{1}(j_{1}), ....f_{k}(i_{k}), ..f_{k}(x).., f_{k}(j_{k})...., f_{n}(i_{n}), ..f_{n}(x).., f_{n}(j_{n}))| = ((j_{1} - i_{1} + 1) + ..(j_{k} - i_{k} + 1).. + (j_{n} - i_{n} + 1))

We see in step 2 of the proof that the requirement is a conjunction of the conditions, proven directly by assumption.

In [42]:
general_len_simple_inst.proof()

step type	requirements	statement
0	instantiation	1, 2, 3	{n in NaturalsPos, (j_{1} - i_{1} + 1) in Naturals, ..(j_{k} - i_{k} + 1) in Naturals.., (j_{n} - i_{n} + 1) in Naturals} |- |(f_{1}(i_{1}), ..f_{1}(x).., f_{1}(j_{1}), ....f_{k}(i_{k}), ..f_{k}(x).., f_{k}(j_{k})...., f_{n}(i_{n}), ..f_{n}(x).., f_{n}(j_{n}))| = ((j_{1} - i_{1} + 1) + ..(j_{k} - i_{k} + 1).. + (j_{n} - i_{n} + 1))
	n : n, f : f, i : i, j : j
1	theorem		|- forall_{n in NaturalsPos} [forall_{f_{1}, ..f_{_a}.., f_{n}, i_{1}, ..i_{_a}.., i_{n}, j_{1}, ..j_{_a}.., j_{n} | (j_{1} - i_{1} + 1) in Naturals, ..(j_{k} - i_{k} + 1) in Naturals.., (j_{n} - i_{n} + 1) in Naturals} (|(f_{1}(i_{1}), ..f_{1}(x).., f_{1}(j_{1}), ....f_{k}(i_{k}), ..f_{k}(x).., f_{k}(j_{k})...., f_{n}(i_{n}), ..f_{n}(x).., f_{n}(j_{n}))| \\  = ((j_{1} - i_{1} + 1) + ..(j_{k} - i_{k} + 1).. + (j_{n} - i_{n} + 1)))]
	proveit.core_expr_types.tuples.general_len
2	assumption		{n in NaturalsPos} |- n in NaturalsPos
3	assumption		{(j_{1} - i_{1} + 1) in Naturals, ..(j_{k} - i_{k} + 1) in Naturals.., (j_{n} - i_{n} + 1) in Naturals} |- ((j_{1} - i_{1} + 1) in Naturals) and ..((j_{k} - i_{k} + 1) in Naturals).. and ((j_{n} - i_{n} + 1) in Naturals)

For a slightly less trivial example, let's do a partial expansion of $f$, $i$, and $j$ within `general_len`.  This time, we will instantiate `general_len` indirectly via the `Len.computation` method.  In fact, it is considered best practice when possible to perform instantiation indirectly via convenient methods and automation routines developed in **Prove-It** theory systems.  This will be discussed at greater length in the "theory systems" chapters.

First some prepartion work:

In [43]:
f_range_of_ranges = general_len.instanceExpr.instanceExpr.lhs.operand[0]

f_range_of_ranges: f_{1}(i_{1}), ..f_{1}(x).., f_{1}(j_{1}), ....f_{k}(i_{k}), ..f_{k}(x).., f_{k}(j_{k})...., f_{n}(i_{n}), ..f_{n}(x).., f_{n}(j_{n})

In [44]:
index_assumptions = general_len.instanceExpr.conditions

index_assumptions: ((j_{1} - i_{1} + 1) in Naturals, ..(j_{k} - i_{k} + 1) in Naturals.., (j_{n} - i_{n} + 1) in Naturals)

In [45]:
Len([n, one]).typical_equiv()

|- |(n, 1)| = |(1, ..k.., 2)|

In [46]:
extended__f_range_of_ranges = ExprTuple(f_range_of_ranges, x)

extended__f_range_of_ranges: (f_{1}(i_{1}), ..f_{1}(x).., f_{1}(j_{1}), ....f_{k}(i_{k}), ..f_{k}(x).., f_{k}(j_{k})...., f_{n}(i_{n}), ..f_{n}(x).., f_{n}(j_{n}), x)

Now we will call the `Len.computation` method to instantiate the `general_len` theorem indirectly.  We pass the relevant `assumptions` into this theory and here we will indicate that we do not want to automatically simplify the resulting length computation so we can see the direct application of theorem in the proof.

In [47]:
Len(extended__f_range_of_ranges).computation(
    assumptions=[*index_assumptions, InSet(n, Naturals)], simplify=False).proof()

|- forall_{i in Naturals, j in Naturals} [forall_{a_{1} in Naturals, ..a_{_a} in Naturals.., a_{i} in Naturals, b in Naturals, c_{1} in Naturals, ..c_{_a} in Naturals.., c_{j} in Naturals | b > 0} ((a_{1} + ..a_{_a}.. + a_{i} + b + c_{1} + ..c_{_a}.. + c_{j}) in NaturalsPos)] {i: 1, j: 0, a: (n,), b: 1, c: ()}


step type	requirements	statement
0	instantiation	1, 2, 3, 4, 5, 6, 7, 8	{n in Naturals, (j_{1} - i_{1} + 1) in Naturals, ..(j_{k} - i_{k} + 1) in Naturals.., (j_{n} - i_{n} + 1) in Naturals} |- |(f_{1}(i_{1}), ..f_{1}(x).., f_{1}(j_{1}), ....f_{k}(i_{k}), ..f_{k}(x).., f_{k}(j_{k})...., f_{n}(i_{n}), ..f_{n}(x).., f_{n}(j_{n}), x)| = ((j_{1} - i_{1} + 1) + ..(j_{k} - i_{k} + 1).. + (j_{n} - i_{n} + 1) + (1 - 1 + 1))
	n : n + 1, f : (x -> f_{1}(x), ..x -> f_{k}(x).., x -> f_{n}(x), _a -> x), i : (i_{1}, ..i_{k}.., i_{n}, 1), j : (j_{1}, ..j_{k}.., j_{n}, 1)
1	theorem		|- forall_{n in NaturalsPos} [forall_{f_{1}, ..f_{_a}.., f_{n}, i_{1}, ..i_{_a}.., i_{n}, j_{1}, ..j_{_a}.., j_{n} | (j_{1} - i_{1} + 1) in Naturals, ..(j_{k} - i_{k} + 1) in Naturals.., (j_{n} - i_{n} + 1) in Naturals} (|(f_{1}(i_{1}), ..f_{1}(x).., f_{1}(j_{1}), ....f_{k}(i_{k}), ..f_{k}(x).., f_{k}(j_{k})...., f_{n}(i_{n}), ..f_{n}(x).., f_{n}(j_{n}))| \\  = ((j_{1} - i_{1} + 1) + ..(j_{k} - i_{k} + 1).. + (j_{n} - i_{n} + 1)))]
	proveit.core_expr_types.tuples.general_len
2	instantiation	9, 26, 10, 11, 12, 15, 13	{n in Naturals} |- (n + 1) in NaturalsPos
	i : 1, j : 0, a : (n), b : 1, c : ()
3	instantiation	14, 15	{n in Naturals} |- |(x -> f_{1}(x), ..x -> f_{a}(x).., x -> f_{n}(x), _a -> x)| = |(1, ..k.., n + 1)|
	f(k) : x -> f_{k}(x), b(_a) : x, i : n
4	instantiation	14, 15	{n in Naturals} |- |(i_{1}, ..i_{a}.., i_{n}, 1)| = |(1, ..k.., n + 1)|
	f(k) : i_{k}, b : 1, i : n
5	instantiation	14, 15	{n in Naturals} |- |(j_{1}, ..j_{a}.., j_{n}, 1)| = |(1, ..k.., n + 1)|
	f(k) : j_{k}, b : 1, i : n
6	assumption		{(j_{1} - i_{1} + 1) in Naturals, ..(j_{k} - i_{k} + 1) in Naturals.., (j_{n} - i_{n} + 1) in Naturals} |- ((j_{1} - i_{1} + 1) in Naturals) and ..((j_{k} - i_{k} + 1) in Naturals).. and ((j_{n} - i_{n} + 1) in Naturals)
7	instantiation	16, 26, 17	|- (1 - 1 + 1) in Naturals
	P(_a) : _a in Naturals, x : 1 - 1 + 1, y : 1
8	instantiation	18	|- (x, ..1 repeats.., x) = (x)
	f(_a) : x, i : 1
9	theorem		|- forall_{i in Naturals, j in Naturals} [forall_{a_{1} in Naturals, ..a_{_a} in Naturals.., a_{i} in Naturals, b in Naturals, c_{1} in Naturals, ..c_{_a} in Naturals.., c_{j} in Naturals | b > 0} ((a_{1} + ..a_{_a}.. + a_{i} + b + c_{1} + ..c_{_a}.. + c_{j}) in NaturalsPos)]
	proveit.number.addition.addNatPosFromNonNeg
10	theorem		|- 0 in Naturals
	proveit.number.sets.integer.zeroInNats
11	instantiation	19	|- |(n)| = |(1, ..k.., 1)|
	a : n
12	theorem		|- |()| = |(1, ..a.., 0)|
	proveit.core_expr_types.tuples.tuple_len_0_typical_equiv
13	instantiation	20, 21	|- 1 > 0
	x : 0, y : 1
14	theorem		|- forall_{f, b} [forall_{i in Naturals} (|(f(1), ..f(a).., f(i), b)| = |(1, ..k.., i + 1)|)]
	proveit.core_expr_types.tuples.extended_range_from1_len_typical_equiv
15	assumption		{n in Naturals} |- n in Naturals
16	theorem		|- forall_{P, x, y | P(y), x = y} P(x)
	proveit.logic.equality.subLeftSideInto
17	instantiation	22, 23	|- (1 - 1 + 1) = 1
	a : 1, b : 1
18	theorem		|- forall_{f, i} ((f(i), ..f(a).., f(i)) = (f(i)))
	proveit.core_expr_types.tuples.singular_range_reduction
19	theorem		|- forall_{a} (|(a)| = |(1, ..k.., 1)|)
	proveit.number.numeral.deci.tuple_len_1_typical_equiv
20	theorem		|- forall_{x, y | x < y} (y > x)
	proveit.number.ordering.reverseLess
21	theorem		|- 0 < 1
	proveit.number.numeral.deci.less_0_1
22	theorem		|- forall_{a in Complexes, b in Complexes} ((a - a + b) = b)
	proveit.number.addition.subtraction.addCancelTriple_12
23	instantiation	24, 25, 26	|- 1 in Complexes
	A : Naturals, B : Complexes, x : 1
24	theorem		|- forall_{A, B | A subseteq B} [forall_{x in A} (x in B)]
	proveit.logic.set_theory.containment.unfoldSubsetEq
25	instantiation	27, 28	|- Naturals subseteq Complexes
	A : Naturals, B : Complexes
26	theorem		|- 1 in Naturals
	proveit.number.numeral.deci.nat1
27	theorem		|- forall_{A, B | A subset B} (A subseteq B)
	proveit.logic.set_theory.containment.relaxSubset
28	theorem		|- Naturals subset Complexes
	proveit.number.sets.complex.natsInCompl

Notice the conjunction of conditions being satisfied in step 6.

Let us be explicit about the algorithm applied when we perform an **instantiation** that eliminates univeral quantifiers, which may involve ranges of parameters.  As we described in the first part of this chapter, an ad-hoc lambda map is created and applied to different parts of the known truth.  This is done to transform assumptions as before.  On the right side of the turnstile, as we eliminate universal quantification, we first create and apply an ad-hoc lambda map for each of its parameter entries (in case there are indices of a parameter range that must be instantiated), then we create and apply an ad-hoc lambda map to each condition (adding those outcomes to our list of *requirements*).  After eliminating all of the desired universal quantifications in that manner, we then create and apply an ad-hoc lambda map to the remaining inner expression.  For the most part, we use the same parameters and operands for each of these ad-hoc lambda maps as before, but the following are exceptions: the ad-hoc lambda map for a parameter entry of an eliminated universal quantifier excludes any entry involving the corresponding parameter variable, and an implicit instantiation is converted to an explicit form after encountering the corresponding explicit range of parameters.

In the instantiation of the `general_len` theorem above, the following ad-hoc lambda maps were created (illustrating both of the exceptions in the changes of the lambda parameters):

$\left(n, f, i, j\right) \mapsto n \in \mathbb{N}^+ \\
\left(n, i, j\right) \mapsto f_{1}, \ldots, f_{n} \\
\left(n, f_{1}, \ldots, f_{n + 1}, j\right) \mapsto i_{1}, \ldots, i_{n} \\
\left(n, f_{1}, \ldots, f_{n + 1}, i_{1}, \ldots, i_{n + 1}\right) \mapsto j_{1}, \ldots, j_{n} \\
\left(n, f_{1}, \ldots, f_{n + 1}, i_{1}, \ldots, i_{n + 1}, j_{1}, \ldots, j_{n + 1}\right) \mapsto \left(\left(j_{1} - i_{1} + 1\right) \in \mathbb{N}\right) \land \ldots \land \left(\left(j_{n} - i_{n} + 1\right) \in \mathbb{N}\right) \\
\left(n, f_{1}, \ldots, f_{n + 1}, i_{1}, \ldots, i_{n + 1}, j_{1}, \ldots, j_{n + 1}\right) \mapsto \begin{array}{c} |\left(f_{1}(i_{1}), \ldots, f_{1}(j_{1}), \ldots\ldots, f_{n}(i_{n}), \ldots, f_{n}(j_{n})\right)| \\  = \left(\left(j_{1} - i_{1} + 1\right) + \ldots + \left(j_{n} - i_{n} + 1\right)\right) \end{array}
$



In [48]:
%end instantiation

# Next chapter: <a href="tutorial04_relabeling.ipynb">ToDo</a>

## <a href="tutorial00_introduction.ipynb#contents">Table of Contents</a>